In [ ]:
import numpy as np
import csv
import pickle

#data loader
   
def onehotencoder(labels,start,stop):
    encode = np.zeros((labels.shape[0],1 - start + stop))
    for i in range(labels.shape[0]):
        encode[i,int(labels[i])] = 1.0
    return encode

def csv2labeldata(stringname):
    with open(stringname, "r") as csvfile:
        reader = csv.reader(csvfile, delimiter=",")
        result = np.array(list(reader)).astype("float")
        labels,data=np.hsplit(result, [1])
        return labels,data 
test_labels,test_data = csv2labeldata("C:/BelgiumTSC/MNIST/MNIST/mnist_dataset/sample_test.csv")
test_labels=onehotencoder(test_labels,0,9)
train_labels,train_data = csv2labeldata("C:/BelgiumTSC/MNIST/MNIST/mnist_dataset/sample_train.csv")
train_labels=onehotencoder(train_labels,0,9)  

import matplotlib.pyplot as plt
X_train_img=np.reshape(train_data,(train_data.shape[0],28,28,1))
plt.imshow(X_train_img[0,:,:,0],cmap = 'gray')
plt.show()

#definite randomize init

#size of neural network
hidden_layer_num = 3
#train_size = 12080
train_size = 7000

test_size = 2019

input_nodes = train_data.shape[1]
h1_nodes = 400
h2_nodes = 200
h3_nodes = 100
output_nodes = 10

#hyperparameters
learning_rate = 0.00000001
#learning_rate = 0.0001
batch = 30

#activation function
def relu(v):
    relu=v
    for i in range(v.shape[0]):
        relu[i]=np.maximum(0,v[i])
    return relu

def relu_prime(v):
    relu=v
    for i in range(v.shape[0]):
        for j in range(v.shape[1]):
            if(v[i][j]>0.0):
                relu[i][j]= 1.0
            else:
                relu[i][j]= 0.0
    return relu
    
def softmax(x):
    prob = np.exp(x-np.max(x))
    p = prob/prob.sum(axis=1)
    return p

np.random.seed(12)
#hidden layer 1
hl1_weights = np.random.randn(input_nodes, h1_nodes)
hl1_weights = hl1_weights/(np.amax(hl1_weights))
hl1_biases = np.zeros((1,h1_nodes))

#hidden layer 2
hl2_weights = np.random.randn(h1_nodes ,h2_nodes)
hl2_weights = hl2_weights/(np.amax(hl2_weights))
hl2_biases = np.zeros((1,h2_nodes))

#hidden layer 3
hl3_weights = np.random.randn(h2_nodes ,h3_nodes)
hl3_weights = hl3_weights/(np.amax(hl3_weights))
hl3_biases = np.zeros((1,h3_nodes))

#output layer
ol_weights = np.random.randn(h3_nodes ,output_nodes)
ol_weights = ol_weights/(np.amax(ol_weights))
ol_biases = np.zeros((1,output_nodes))

#actual model param tuple
weights_and_biases = (hl1_weights,hl1_biases, hl2_weights,hl2_biases, hl3_weights,hl3_biases, ol_weights,ol_biases)

def feedforward(input_data,weights_and_biases):
    hl1_weights,hl1_biases, hl2_weights,hl2_biases, hl3_weights,hl3_biases, ol_weights,ol_biases = weights_and_biases

    z1 = relu(np.dot(input_data ,hl1_weights) + hl1_biases)
    z2 = relu(np.dot(z1 ,hl2_weights) + hl2_biases)
    z3 = relu(np.dot(z2, hl3_weights) + hl3_biases)
    
    return (softmax(np.dot(z3 ,ol_weights) + ol_biases),z1,z2,z3)
    #return (relu(np.dot(z3, ol_weights) + ol_biases),z1,z2,z3)
def backpropogate(input_data,actual_label,probabilistic_value ,weights_and_biases, alpha):
    pred,z1,z2,z3 = probabilistic_value
    hl1_weights,hl1_biases, hl2_weights,hl2_biases, hl3_weights,hl3_biases, ol_weights,ol_biases = weights_and_biases
    
    delta4 = (pred - actual_label)  #predict - label
    dow = np.dot(z3.T,delta4)
    dob = np.sum(delta4)
    delta3 = np.multiply(np.dot(delta4,ol_weights.T), relu_prime(z3))
    
    dhlw_3 = np.dot(z2.T,delta3) 
    dhlb_3 = np.sum(delta3)
    
    delta2 = np.multiply(np.dot(delta3,hl3_weights.T) , relu_prime(z2))
    
    dhlw_2 = np.dot(z1.T,delta2) 
    dhlb_2 = np.sum(delta2)

    delta1 = np.multiply(np.dot(delta2,hl2_weights.T) , relu_prime(z1))
    
    dhlw_1 = np.dot(input_data.T,delta1) 
    dhlb_1 = np.sum(delta1)

    #regularization and
    #updating
    dhlw_1 += 0.01 * hl1_weights
    hl1_weights -= alpha*dhlw_1
    hl1_biases -= alpha*dhlb_1
    
    dhlw_2 += 0.01 * hl2_weights
    hl2_weights -= alpha*dhlw_2
    hl2_biases -= alpha*dhlb_2
    
    dhlw_3 += 0.01 * hl3_weights
    hl3_weights -= alpha*dhlw_3
    hl3_biases -= alpha*dhlb_3
    
    dow += 0.01 * ol_weights
    ol_weights -= alpha*dow
    ol_biases -= alpha*dob
    
    return (hl1_weights,hl1_biases, hl2_weights,hl2_biases, hl3_weights,hl3_biases, ol_weights,ol_biases)


def backpropogate_adam(adam_parameters, input_data,actual_label,probabilistic_value ,weights_and_biases, alpha, decay_rate_1 = None, 
              decay_rate_2 = None, epsilon = None):
    pred,z1,z2,z3 = probabilistic_value
    hl1_weights,hl1_biases, hl2_weights,hl2_biases, hl3_weights,hl3_biases, ol_weights,ol_biases = weights_and_biases
    dow_m, dow_v,dob_m, dob_v, dhl3_m, dhl3_v, dhlb3_m, dhlb3_v, dhl2_m, dhl2_v, dhlb2_m, dhlb2_v, dhl1_m, dhl1_v, dhlb1_m, dhlb1_v, t = adam_parameters
    
    delta4 = (pred - actual_label)  #predict - label
    dow = np.dot(z3.T,delta4)
    dob = np.sum(delta4)

    delta3 = np.multiply(np.dot(delta4,ol_weights.T), relu_prime(z3))
    
    dhlw_3 = np.dot(z2.T,delta3) 
    dhlb_3 = np.sum(delta3)
    
    delta2 = np.multiply(np.dot(delta3,hl3_weights.T) , relu_prime(z2))
    
    dhlw_2 = np.dot(z1.T,delta2) 
    dhlb_2 = np.sum(delta2)

    delta1 = np.multiply(np.dot(delta2,hl2_weights.T) , relu_prime(z1))
    
    dhlw_1 = np.dot(input_data.T,delta1) 
    dhlb_1 = np.sum(delta1)

    #regularization and
    #updating
    dhlw_1 += 0.01 * hl1_weights  
    dhlw_2 += 0.01 * hl2_weights    
    dhlw_3 += 0.01 * hl3_weights
    dow += 0.01 * ol_weights
          
    t =t + 1 # Increment Time Step
            
    # Computing 1st and 2nd moment for each layer
    dow_m = dow_m * decay_rate_1 + (1- decay_rate_1) * dow
    dob_m = dob_m * decay_rate_1 + (1- decay_rate_1) * dob
    
    dhl3_m = dhl3_m * decay_rate_1 + (1- decay_rate_1) * dhlw_3
    dhlb3_m = dhlb3_m * decay_rate_1 + (1- decay_rate_1) * dhlb_3
    
    dhl2_m = dhl2_m * decay_rate_1 + (1- decay_rate_1) * dhlw_2
    dhlb2_m = dhlb2_m * decay_rate_1 + (1- decay_rate_1) * dhlb_2
    
    dhl1_m = dhl1_m * decay_rate_1 + (1- decay_rate_1) * dhlw_1
    dhlb1_m = dhlb1_m * decay_rate_1 + (1- decay_rate_1) * dhlb_1
    
    
    dow_v = dow_v * decay_rate_2 + (1- decay_rate_2) * (dow ** 2)
    dob_v = dob_v * decay_rate_2 + (1- decay_rate_2) * (dob ** 2)
    
    dhl3_v = dhl3_v * decay_rate_2 + (1- decay_rate_2) * (dhlw_3 ** 2)
    dhlb3_v = dhlb3_v * decay_rate_2 + (1- decay_rate_2) * (dhlb_3 ** 2)
    
    dhl2_v = dhl2_v * decay_rate_2 + (1- decay_rate_2) * (dhlw_2 ** 2)
    dhlb2_v = dhlb2_v * decay_rate_2 + (1- decay_rate_2) * (dhlb_2 ** 2)
    
    dhl1_v = dhl1_v * decay_rate_2 + (1- decay_rate_2) * (dhlw_1 ** 2)
    dhlb1_v = dhlb1_v * decay_rate_2 + (1- decay_rate_2) * (dhlb_1 ** 2)
            
    dow_m_corrected  = dow_m/(1-(decay_rate_1 ** t))
    dow_v_corrected  = dow_v/(1-(decay_rate_2 ** t))
    dob_m_corrected  = dob_m/(1-(decay_rate_1 ** t))
    dob_v_corrected  = dob_v/(1-(decay_rate_2 ** t))
    
    dhl3_m_corrected  = dhl3_m/(1-(decay_rate_1 ** t))
    dhl3_v_corrected  = dhl3_v/(1-(decay_rate_2 ** t))
    dhlb3_m_corrected  = dhlb3_m/(1-(decay_rate_1 ** t))
    dhlb3_v_corrected  = dhlb3_v/(1-(decay_rate_2 ** t))

    dhl2_m_corrected  = dhl2_m/(1-(decay_rate_1 ** t))
    dhl2_v_corrected  = dhl2_v/(1-(decay_rate_2 ** t))
    dhlb2_m_corrected  = dhlb2_m/(1-(decay_rate_1 ** t))
    dhlb2_v_corrected  = dhlb2_v/(1-(decay_rate_2 ** t))

    dhl1_m_corrected  = dhl1_m/(1-(decay_rate_1 ** t))
    dhl1_v_corrected  = dhl1_v/(1-(decay_rate_2 ** t))
    dhlb1_m_corrected  = dhlb1_m/(1-(decay_rate_1 ** t))
    dhlb1_v_corrected  = dhlb1_v/(1-(decay_rate_2 ** t))
              
    # Update Weights
    dow = dow_m_corrected / (np.sqrt(dow_v_corrected) + epsilon)
    dob = dob_m_corrected / (np.sqrt(dob_v_corrected) + epsilon)
    dhlw_3 = dhl3_m_corrected / (np.sqrt(dhl3_v_corrected) + epsilon)
    dhlb_3 = dhlb3_m_corrected / (np.sqrt(dhlb3_v_corrected) + epsilon)
    dhlw_2 = dhl2_m_corrected / (np.sqrt(dhl2_v_corrected) + epsilon)
    dhlb_2 = dhlb2_m_corrected / (np.sqrt(dhlb2_v_corrected) + epsilon)
    dhlw_1 = dhl1_m_corrected / (np.sqrt(dhl1_v_corrected) + epsilon)
    dhlb_1 = dhlb1_m_corrected / (np.sqrt(dhlb1_v_corrected) + epsilon)
            
    hl1_weights -= alpha*dhlw_1
    hl1_biases -= alpha*dhlb_1
    
    hl2_weights -= alpha*dhlw_2
    hl2_biases -= alpha*dhlb_2
    
    hl3_weights -= alpha*dhlw_3
    hl3_biases -= alpha*dhlb_3
    
    ol_weights -= alpha*dow
    ol_biases -= alpha*dob
    weights_and_biases = (hl1_weights,hl1_biases, hl2_weights,hl2_biases, hl3_weights,hl3_biases, ol_weights,ol_biases)
    adam_parameters = (dow_m, dow_v,dob_m, dob_v, dhl3_m, dhl3_v, dhlb3_m, dhlb3_v, dhl2_m, dhl2_v, dhlb2_m, dhlb2_v, dhl1_m, dhl1_v, dhlb1_m, dhlb1_v, t)
    
    return (weights_and_biases, adam_parameters)

#def train_network(input_data,input_label,weights_and_biases,alpha, adam_parameters):
def train_network(input_data,input_label,weights_and_biases,alpha):    
    #feedforward
    initial_values=feedforward(input_data,weights_and_biases)   
    
    #backpropagation    
    weights_and_biases=backpropogate(input_data,input_label,initial_values,weights_and_biases,alpha)    
    #weights_and_biases, adam_parameters=backpropogate_adam(adam_parameters, input_data,input_label,initial_values,weights_and_biases,alpha,
    #                                    decay_rate_1 = 0.9,
    #                                      decay_rate_2 = 0.99,
    #                                      epsilon = 10e-8)    
    #return (weights_and_biases,initial_values[0], adam_parameters)
    return (weights_and_biases,initial_values[0])

def prediction(input_data,weights_and_biases):
    return feedforward(input_data,weights_and_biases)[0]
    
def accuracy(data,labels,weights_and_biases,size_data):
    total = 0
    for i in range(size_data):
        if(np.argmax(prediction(data[i],weights_and_biases)) == np.argmax(labels[i])):
            total += 1
    return ((100.0 * total)/size_data)

def accuracy_end(data,labels,weights_and_biases,size_data):
    total = 0
    value = []
    for i in range(size_data):
        value.append(prediction(data[i],weights_and_biases))
        if(np.argmax(value) == np.argmax(labels[i])):
            total += 1
    return (value)

def residuals(data,labels,weights_and_biases,size_data):
    total = 0
    for i in range(size_data):
        total+=((prediction(data[i],weights_and_biases) - labels[i])** 2.0).mean(axis=1)
    return ((100.0 * total)/size_data)

accuracy_list = []
residuals_list = []

dow_m = 0
dow_v = 0
dob_m = 0
dob_v = 0
dhl3_m = 0
dhl3_v = 0
dhlb3_m = 0
dhlb3_v = 0
dhl2_m = 0
dhl2_v = 0
dhlb2_m = 0
dhlb2_v = 0
dhl1_m = 0
dhl1_v = 0
dhlb1_m = 0
dhlb1_v = 0
t = 0

adam_parameters = (dow_m, dow_v, dob_m, dob_v, dhl3_m, dhl3_v, dhlb3_m, dhlb3_v, dhl2_m, dhl2_v, dhlb2_m, 
                       dhlb2_v, dhl1_m, dhl1_v, dhlb1_m, dhlb1_v, t)
for j in range(train_size):
    for i in range(batch):
        training = train_data[j].reshape((1,train_data[j].shape[0]))
        #weights_and_biases,predict, adam_parameters = train_network(training,train_labels[j],weights_and_biases,learning_rate, adam_parameters)
        weights_and_biases,predict = train_network(training,train_labels[j],weights_and_biases,learning_rate)
        tr_label = train_labels[j].reshape((1,train_labels[j].shape[0]))
        if (residuals(training,tr_label,weights_and_biases,1)==0.0):
            break        

    if j%1000==0:
       
        #print('\nTraining accuracy for current:',accuracy(training,tr_label,weights_and_biases,1),"%")       
        acc = accuracy(test_data,test_labels,weights_and_biases,test_size)
        print('\nTest accuracy:',acc,"%")
        accuracy_list.append(acc)
        #print('\nTraining error:',residuals(training,tr_label,weights_and_biases,1))
        res = residuals(test_data,test_labels,weights_and_biases,test_size)
        print('\nTest error:',res)
        residuals_list.append(res)

plt.plot(res)
plt.show()
value = accuracy_end(test_data,test_labels,weights_and_biases,test_size)
